In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.svm import LinearSVC
import time as t
import random

In [2]:
def get_renamed_labels( X ):
    return np.cumprod( np.flip( 2 * X - 1 , axis = 1 ), axis = 1 )

def add_bias(X):
    tmp = np.ones(X.shape[0])
    tmp = np.resize(tmp, (X.shape[0], 1)) 
    X = np.hstack((X,tmp))
    return X

def get_features(X):
    tmp = []
    for i in range(X.shape[0]):
        a = X[i]
        l = []
        for j in range(9):
            for k in range(9):
                l.extend(a*X[i][k]*X[i][j])
        tmp.append(l)
    return tmp
    

In [3]:
data = np.loadtxt( "secret_train.dat" )
data_test = np.loadtxt('test.dat')
# data_secret_train = np.loadtxt('secret_train.dat')

In [4]:
x_train = data[:,:-1]
y_train = data[:,-1]
x_train = get_renamed_labels(x_train)
x_train = add_bias(x_train)
x_train = np.array(get_features(x_train))


In [5]:
x_test = data_test[:,:-1]
y_test = data_test[:,-1]
x_test = get_renamed_labels(x_test)
x_test = add_bias(x_test)
x_test = np.array(get_features(x_test))

In [6]:
y_train = np.where( y_train > 0, 1, -1 )
y_test = np.where( y_test > 0, 1, -1 )

In [7]:
def solver(epochs):
    global w,b,eta
    for i in range(epochs):
        #randperm = np.random.permutation(y_train.size)
        #j = random.randrange(0,y_train.shape[0] - 1)
        j = i%(x_train.shape[0])
        x = x_train[j,:]
        y = y_train[j]
        if 1 - np.multiply(x.dot(w),y) <= 0:
            gradient = w
        else:
            gradient = w-C*x*y
        learning_rate = stepLengthGenerator('linear',eta)(i+1)
        w = w-learning_rate*gradient
        #b = b-learning_rate*gradient
        
def mySVM( X ):
    return X.dot(w)

def stepLengthGenerator( mode, eta ):
    if mode == "constant":
        return lambda t: eta
    elif mode == "linear":
        return lambda t: eta/(t+1)
    elif mode == "quadratic":
        return lambda t: eta/np.sqrt(t+1)

In [8]:
eta = 0.5
C = 5
w = np.ones(  x_train.shape[1] )
#solver(0.0005,100000)
solver(10000)



In [9]:
y_pred = mySVM( x_test )

y_pred = np.where( y_pred > 0, 1, -1 )

print( np.average( y_test == y_pred ) )

0.94335


In [10]:
data = np.loadtxt('secret_test.dat')
x_secret_test = data[:,:-1]
y_secret_test = data[:,-1]

In [11]:
x_secret_test = get_renamed_labels(x_secret_test)
x_secret_test = add_bias(x_secret_test)
x_secret_test = np.array(get_features(x_secret_test))

In [12]:
y_secret_test = np.where(y_secret_test>0,1,-1)

In [13]:
y_pred = mySVM( x_secret_test )

y_pred = np.where( y_pred > 0, 1, -1 )

print( np.average( y_secret_test == y_pred ) )

0.9411
